# Ataxx Zero Transformer - Colab/Kaggle Notebook (Pro)

This notebook is a production-style workflow to train and resume **Ataxx Zero** with:
- reproducible setup with `uv`,
- optional Hugging Face checkpoint uploads,
- clean logging for hosted notebooks,
- safe restart/resume flow.


## 0) Runtime Recommendations

1. Use **GPU runtime** when possible.
2. In Colab: Runtime -> Change runtime type -> GPU.
3. In Kaggle: Settings -> Accelerator -> GPU.
4. If you restart runtime, re-run all cells from top.


In [ ]:
#@title 1) Experiment Parameters
GITHUB_REPO = "https://github.com/YOUR_USERNAME/ataxx-zero.git"  #@param {type:"string"}
GIT_BRANCH = "main"  #@param {type:"string"}
WORKDIR = "/content/ataxx-zero"  # Colab default path

# Training profile presets for notebook GPUs
PROFILE = "balanced"  #@param ["debug", "balanced", "strong"]

# Hugging Face integration
ENABLE_HF = False  #@param {type:"boolean"}
HF_REPO_ID = "YOUR_USERNAME/ataxx-zero-transformer"  #@param {type:"string"}
HF_TOKEN = ""  #@param {type:"string"}

# Optional: stop early at this iteration in notebook session
MAX_ITERS_OVERRIDE = 0  #@param {type:"integer"}


In [ ]:
#@title 2) Clone / Update Repository
import os
from pathlib import Path

repo_path = Path(WORKDIR)

if not repo_path.exists():
    !git clone {GITHUB_REPO} {WORKDIR}
else:
    print(f"Repo already exists at {WORKDIR}")

%cd {WORKDIR}
!git fetch --all --prune
!git checkout {GIT_BRANCH}
!git pull --ff-only
!git status --short


In [ ]:
#@title 3) Install uv + Sync Dependencies
import os

# Install uv if needed
!python -m pip -q install uv

# Use project Python version + dependencies
!uv sync

# Show key tool versions
!uv --version
!uv run python --version
!uv run python -c "import torch; print('torch', torch.__version__, 'cuda', torch.cuda.is_available())"


In [ ]:
#@title 4) Quick Project Health Check
!uv run ruff check src train_improved.py scripts/play_pygame.py
!uv run pyrefly check src


## 5) Configure Training for Notebook Session

This cell updates `train_improved.CONFIG` in-memory and runs training directly.
No file edits required.


In [ ]:
#@title 5) Build Notebook Training Config
import torch
import train_improved as t

# Compact profiles for hosted notebook sessions
profiles = {
    "debug": {
        "iterations": 3,
        "episodes_per_iter": 12,
        "mcts_sims": 64,
        "epochs": 2,
        "batch_size": 64,
        "d_model": 96,
        "nhead": 8,
        "num_layers": 3,
        "dim_feedforward": 256,
    },
    "balanced": {
        "iterations": 12,
        "episodes_per_iter": 40,
        "mcts_sims": 160,
        "epochs": 4,
        "batch_size": 96,
        "d_model": 128,
        "nhead": 8,
        "num_layers": 4,
        "dim_feedforward": 384,
    },
    "strong": {
        "iterations": 20,
        "episodes_per_iter": 70,
        "mcts_sims": 280,
        "epochs": 6,
        "batch_size": 128,
        "d_model": 192,
        "nhead": 8,
        "num_layers": 6,
        "dim_feedforward": 512,
    },
}

cfg = profiles[PROFILE].copy()

# Common notebook-friendly defaults
cfg.update({
    "verbose_logs": False,
    "episode_log_every": 50,
    "save_every": 2,
    "val_split": 0.1,
    "seed": 42,
    "checkpoint_dir": "checkpoints",
    "log_dir": "logs",
    "onnx_path": "ataxx_model.onnx",
})

if MAX_ITERS_OVERRIDE > 0:
    cfg["iterations"] = int(MAX_ITERS_OVERRIDE)

# HF checkpointing integration
if ENABLE_HF:
    cfg["hf_enabled"] = True
    cfg["hf_repo_id"] = HF_REPO_ID
    cfg["hf_token_env"] = "HF_TOKEN"
    cfg["hf_local_dir"] = "hf_checkpoints"
    cfg["keep_last_n_hf_checkpoints"] = 3
else:
    cfg["hf_enabled"] = False

# Push into project config dict
for k, v in cfg.items():
    t.CONFIG[k] = v

# Export HF token if provided
if ENABLE_HF and HF_TOKEN.strip():
    import os
    os.environ["HF_TOKEN"] = HF_TOKEN.strip()

print("Active CONFIG subset:")
for key in [
    "iterations", "episodes_per_iter", "mcts_sims", "epochs", "batch_size",
    "d_model", "num_layers", "dim_feedforward", "save_every",
    "hf_enabled", "hf_repo_id"
]:
    print(f"  {key}: {t.CONFIG.get(key)}")

print("
GPU:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))


In [ ]:
#@title 6) Start / Resume Training
# If HF is enabled and checkpoints exist, train_improved will auto-resume from latest HF checkpoint.
import train_improved as t

t.main()


In [ ]:
#@title 7) Inspect Outputs
from pathlib import Path

for p in [Path("checkpoints"), Path("logs"), Path("hf_checkpoints")]:
    if p.exists():
        print(f"
{p}/")
        for f in sorted(p.glob("*"))[:30]:
            print("  -", f.name)


In [ ]:
#@title 8) TensorBoard (optional)
# In Colab, run this cell after a few iterations.
%load_ext tensorboard
%tensorboard --logdir logs


## 9) Optional: Play Against AI (local desktop only)

`pygame` UI is for local environment with display.
In Colab/Kaggle notebook kernels, interactive window rendering is generally unavailable.

Local command examples:
- `uv run python scripts/play_pygame.py --opponent heuristic --heuristic-level easy`
- `uv run python scripts/play_pygame.py --opponent model --checkpoint checkpoints/manual_iter_002.ckpt`


## 10) Troubleshooting

- **OOM (GPU out of memory)**: reduce `batch_size`, `d_model`, `num_layers`, or `mcts_sims`.
- **Too slow**: lower `episodes_per_iter` and `mcts_sims`; use `PROFILE=debug` first.
- **HF upload fails**: confirm `ENABLE_HF=True`, valid `HF_REPO_ID`, and `HF_TOKEN`.
- **No resume**: verify files exist in `hf_checkpoints/` or HF repo model files.
